In [1]:
import numpy as np
import cv2

In [2]:
img=cv2.imread("imori.jpg").astype(np.float32)

In [3]:
def BGR2GRAY(img):
    b=img[:,:,0].copy()
    g=img[:,:,1].copy()
    r=img[:,:,2].copy()

    out=0.2126*r+0.7152*g+0.0722*b
    out=out.astype(np.uint8)

    return out

def gaussian_filter(img,K_size=3,sigma=1.3):
    if len(img.shape)==3:
        H,W,C=img.shape
        gray=False
    else:
        img=np.expand_dims(img,axis=-1)
        H,W,C=img.shape
        gray=True

    # Zero padding
    pad=K_size//2
    out=np.zeros([H+pad*2,W+pad*2,C],dtype=np.float)
    out[pad:pad+H,pad:pad+W]=img.copy().astype(np.float)

    K=np.zeros((K_size,K_size),dtype=np.float)
    for x in range(-pad,-pad+K_size):
        for y in range(-pad,-pad+K_size):
            K[y+pad,x+pad]=np.exp(-(x**2+y**2)/(2*(sigma**2)))
    K/=(2*np.pi*sigma*sigma)
    K/=K.sum()

    tmp=out.copy()

    # filtering
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad+y,pad+x,c]=np.sum(K*tmp[y:y+K_size,x:x+K_size,c])


    out=np.clip(out,0,255)
    out=out[pad:pad+H,pad:pad+W].astype(np.uint8)

    if gray:
        out=out[...,0]

    return out

# soble filter
def sobel_filter(img,K_size=3):
    if len(img.shape)==3:
        H,W,C=img.shape
    else:
        H,W=img.shape

    pad=K_size//2
    out=np.zeros((H+pad*2,W+pad*2),dtype=np.float)
    out[pad:pad+H,pad:pad+W]=img.copy().astype(np.float)
    tmp=out.copy()

    out_v=out.copy()
    out_h=out.copy()

    # sobel vertical
    kv=[[1.,2.,1.],[0.,0.,0.],[-1.,-2.,-1.]]
    # sobel horizontal
    kh=[[1.,0.,-1.],[2.,0.,-2.],[1.,0.,-1.]]

    # filtering
    for y in range(H):
        for x in range(W):
            out_v[pad+y,pad+x]=np.sum(kv*(tmp[y:y+K_size,x:x+K_size]))
            out_h[pad+y,pad+x]=np.sum(kh*(tmp[y:y+K_size,x:x+K_size]))

    out_v=np.clip(out_v,0,255)
    out_h=np.clip(out_h,0,255)

    out_v=out_v[pad:pad+H,pad:pad+W].astype(np.uint8)
    out_h=out_h[pad:pad+H,pad:pad+W].astype(np.uint8)

    return out_v,out_h

def get_edge_angle(fx,fy):
    edge=np.sqrt(np.power(fx.astype(np.float32),2)+np.power(fy.astype(np.float32),2))
    edge=np.clip(edge,0,255)
    fx=np.maximum(fx,1e-5)

    angle=np.arctan(fy/fx)

    return edge,angle

def angle_quantization(angle):
    angle=angle/np.pi*180
    angle[angle<-22.5]=180+angle[angle<-22.5]
    _angle=np.zeros_like(angle,dtype=np.uint8)
    _angle[np.where(angle<=22.5)]=0
    _angle[np.where((angle>22.5) & (angle<=67.5))]=45
    _angle[np.where((angle>67.5) & (angle<=112.5))]=90
    _angle[np.where((angle>112.5) & (angle<=157.5))]=135

    return _angle

In [4]:
def Canny_step1(img):
    
    gray=BGR2GRAY(img)
    gaussian=gaussian_filter(gray,K_size=5,sigma=1.4)
    fy,fx=sobel_filter(gaussian,K_size=3)
    edge,angle=get_edge_angle(fx,fy)
    angle=angle_quantization(angle)
    
    return edge,angle

In [5]:
ans1=img.copy()
edge,angle=Canny_step1(ans1)

edge=edge.astype(np.uint8)
angle=angle.astype(np.uint8)

print(cv2.imwrite("ans1edge.jpg",edge))
print(cv2.imwrite("ans1angle.jpg",angle))

True
True


In [6]:
def non_maximum_suppression(angle,edge):
    H,W=angle.shape
    _edge=edge.copy()
    
    for y in range(H):
        for x in range(W):
            if angle[y,x]==0:
                delta=[-1,0,1,0] # dx1,dy1,dx2,dy2
            elif angle[y,x]==45:
                delta=[-1,1,1,-1]
            elif angle[y,x]==90:
                delta=[0,-1,0,1]
            elif angle[y,x]==135:
                delta=[-1,-1,1,1]
            
            if x==0:
                delta[0]=max(delta[0],0)
                delta[2]=max(delta[2],0)
            if x==W-1:
                delta[0]=min(delta[0],0)
                delta[2]=min(delta[2],0)
            if y==0:
                delta[1]=max(delta[1],0)
                delta[3]=max(delta[3],0)
            if y==H-1:
                delta[1]=min(delta[1],0)
                delta[3]=min(delta[3],0)
            if max(max(edge[y,x],edge[y+delta[1],x+delta[0]]),edge[y+delta[3],x+delta[2]])!=edge[y,x]:
                _edge[y,x]=0
    
    return _edge

In [7]:
ans2=img.copy().astype(np.float32)
edge2,angle2=Canny_step1(ans2)
_edge=non_maximum_suppression(angle2,edge2).astype(np.uint8)
_edge=np.clip(_edge,0,255)
print(cv2.imwrite("ans2edge.jpg",_edge))

True


In [8]:
def hysterisis(edge,HT=100,LT=30):
    H,W=edge.shape
    
    edge[edge>=HT]=255
    edge[edge<=LT]=0
    
    _edge=np.zeros((H+2,W+2),dtype=np.float32)
    _edge[1:H+1,1:W+1]=edge
    
    nn=np.array(((1.,1.,1.),(1.,0.,1.),(1.,1.,1.)),dtype=np.float32)
    
    for y in range(1,H+2):
        for x in range(1,W+2):
            if _edge[y,x]<LT or _edge[y,x]>HT:
                continue
            if np.max(_edge[y-1:y+2,x-1:x+2]*nn) >= HT:
                _edge[y,x]=255
            else:
                _edge[y,x]=0
    
    edge=_edge[1:H+1,1:W+1]
    
    return edge

def Canny(img):
    edge,angle=Canny_step1(img)
    edge=non_maximum_suppression(angle,edge)
    
    out=hysterisis(edge,50,20)
    
    return out

In [14]:
ans3=img.copy().astype(np.float32)
ans3=Canny(ans3).astype(np.uint8)
print(cv2.imwrite("ans3.jpg",ans3))

True


In [15]:
def voting(edge):
    H,W=edge.shape
    drho=1
    dtheta=1

    # get rho max length
    rho_max=np.ceil(np.sqrt(H**2+W**2)).astype(np.int)

    # hough table
    hough=np.zeros((rho_max*2,180),dtype=np.int)
    
    # get index of edge
    ind=np.where(edge==255)
    
    # hough transformation
    for y,x in zip(ind[0],ind[1]):
        for theta in range(0,180,dtheta):
            t=np.pi/180*theta
            rho=int(x*np.cos(t)+y*np.sin(t))
            
            # vote
            hough[rho+rho_max,theta]+=1
            
    out=hough.astype(np.uint8)
    
    return out

def Hough_Line_step1(edge):
    return voting(edge)

In [16]:
thorino_img=cv2.imread("./thorino.jpg").astype(np.float32)
edge_thorino=Canny(thorino_img)
ans4=Hough_Line_step1(edge_thorino).astype(np.uint8)
cv2.imwrite("ans4.jpg",ans4)

True

In [30]:
def non_maximum_suppression_for_hough(hough):
    rho_max,_=hough.shape
    
    for y in range(rho_max):
        for x in range(180):
            x1=max(x-1,0)
            x2=min(x+2,180)
            y1=max(y-1,0)
            y2=min(y+2,rho_max)
            if np.max(hough[y1:y2,x1:x2])==hough[y,x] and hough[y,x]!=0:
                pass
            else:
                hough[y,x]=0
                
    # for hough visualization, get top-10 x index of hough table
    ind_x=np.argsort(hough.ravel())[::-1][:20]
    # get y index
    ind_y=ind_x.copy()
    thetas=ind_x%180
    rhos=ind_y//180
    _hough=np.zeros_like(hough,dtype=np.int)
    _hough[rhos,thetas]=255
    
    return _hough

In [31]:
ans5=non_maximum_suppression_for_hough(ans4.copy())
cv2.imwrite("ans5.jpg",ans5.astype(np.uint8))

True

In [46]:
def non_maxium_suppression_for_inversehough(hough):
    rho_max,_=hough.shape

    for y in range(rho_max):
        for x in range(180):
            x1=max(x-1,0)
            x2=min(x+2,180)
            y1=max(y-1,0)
            y2=min(y+2,rho_max)
            if np.max(hough[y1:y2,x1:x2])==hough[y,x] and hough[y,x]!=0:
                pass
            else:
                hough[y,x]=0
    return hough

def inverse_hough(hough,img):                    
    H,W,_=img.shape
    rho_max,_=hough.shape
    
    out=img.copy()
    
    # for hough visualization, get top-10 x index of hough table
    ind_x=np.argsort(hough.ravel())[::-1][:20]
    # get y index
    ind_y=ind_x.copy()
    thetas=ind_x%180
    rhos=ind_y//180-rho_max/2
    
    # each theta and rho
    for theta,rho in zip(thetas,rhos):
        t=np.pi/180.*theta
        
        # hough -> (x,y)
        for x in range(W):
            if np.sin(t)!=0:
                y=-(np.cos(t)/np.sin(t)*x+(rho)/np.sin(t))
                y=int(y)
                if y>=H or y<0:
                    continue
                out[y,x]=[0,0,255]
        for y in range(H):
            if np.cos(t)!=0:
                x=-(np.sin(t)/np.cos(t))*y+(rho)/np.cos(t)
                x=int(x)
                if x>=W or x<0:
                    continue
                out[y,x]=[0,0,255]
    
    out=out.astype(np.uint8)
    return out
                
def Hough_Line(img,hough):
    suppressed=non_maxium_suppression_for_inversehough(hough)
    return inverse_hough(suppressed,img)

In [47]:
ans6=Hough_Line(thorino_img,ans4.copy()).astype(np.uint8)
cv2.imwrite("ans6.jpg",ans6)

True

In [57]:
def otsu_binarization(img, th=128):
    H, W = img.shape
    out = img.copy()

    max_sigma = 0
    max_t = 0

    # determine threshold
    for _t in range(1, 255):
        v0 = out[np.where(out < _t)]
        m0 = np.mean(v0) if len(v0) > 0 else 0.
        w0 = len(v0) / (H * W)
        v1 = out[np.where(out >= _t)]
        m1 = np.mean(v1) if len(v1) > 0 else 0.
        w1 = len(v1) / (H * W)
        sigma = w0 * w1 * ((m0 - m1) ** 2)
        if sigma > max_sigma:
            max_sigma = sigma
            max_t = _t

    # Binarization
    print("threshold >>", max_t)
    th = max_t
    out[out < th] = 0
    out[out >= th] = 255

    return out

def MorphologyDilate(gray_img,Dil_time=1):
    H,W=gray_img.shape
    output=np.zeros((H+2,W+2),dtype=np.uint8)
    tmp=output.copy()
    tmp[1:1+H,1:1+W]=gray_img
    
    K=[[0,1,0],[1,0,1],[0,1,0]]
    for y in range(H):
        for x in range(W):
            if tmp[y+1,x+1]==255:
                output[y+1,x+1]=255
            elif np.sum((tmp[y:y+3,x:x+3]*K).ravel())>=255:
                output[y+1,x+1]=255
                
    Dil_time-=1
    if Dil_time == 0:
        return output[1:1+H,1:1+W]
    else:
        return MorphologyDilate(output[1:1+H,1:1+W],Dil_time)

In [58]:
ans7=img.copy().astype(np.float32)
ans7=otsu_binarization(BGR2GRAY(ans7))
ans7=MorphologyDilate(ans7,Dil_time=2)
cv2.imwrite("ans7.jpg",ans7)

threshold >> 127


True

In [80]:
def MorphologyErosion(gray_img,Ero_time=1):
    H,W=gray_img.shape
    gray_img=np.pad(gray_img,(1,1),'edge')
    output=np.zeros_like(gray_img,dtype=np.uint8)
    
    K=np.array(((0,1,0),(1,0,1),(0,1,0)),dtype=np.int)
    for y in range(H):
        for x in range(W):
            if gray_img[y+1,x+1]==255:
                if np.sum((gray_img[y:y+3,x:x+3]*K).ravel())>=(255*4):
                    output[y+1,x+1]=255
    
    Ero_time-=1
    if Ero_time==0:
        return output[1:1+H,1:1+W]
    else:
        return MorphologyErosion(output[1:1+H,1:1+W],Ero_time)

In [82]:
ans8=img.copy().astype(np.float32)
ans8=otsu_binarization(BGR2GRAY(ans8))
ans8=MorphologyErosion(ans8,Ero_time=2)
cv2.imwrite("ans8.jpg",ans8)

threshold >> 127


True

In [83]:
def Morphology_Opening(gray_img,times=1):
    tmp=MorphologyErosion(gray_img,times)
    output=MorphologyDilate(tmp,times)
    
    return output

In [84]:
ans9=img.copy().astype(np.float32)
ans9=otsu_binarization(BGR2GRAY(ans9))
ans9=Morphology_Opening(ans9,times=1)
cv2.imwrite("ans9.jpg",ans9)

threshold >> 127


True

In [85]:
def Morphology_Closing(gray_img,times=1):
    tmp=MorphologyDilate(gray_img,times)
    output=MorphologyErosion(tmp,times)
    
    return output

In [86]:
ans10=img.copy().astype(np.float32)
ans10=Canny(ans10).astype(np.uint8)
ans10=Morphology_Closing(ans10)
cv2.imwrite("ans10.jpg",ans10)

True